In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import chromedriver_binary
from pymongo import MongoClient 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
% matplotlib inline

In [2]:
# making connection to mongodb

client = MongoClient()
database_schoolFinds = client['db_schoolFinds']
collection_schoolsFindsPass = database_schoolFinds['collection_schoolFindsPass']
collection_schoolsFindsFail = database_schoolFinds['collection_schoolFindsFail']

In [4]:
driver = webdriver.Chrome()
driver.get("http://www.schoolfinds.com")

In [5]:
driver.maximize_window()

In [6]:
selectCityButton = driver.find_element_by_xpath("//div[@class='header-content visible-desktop']//div[@class='span4']//a[2]")
selectCityButton.click()
sleep(2)

allCityLinks = driver.find_elements_by_xpath("//div[@id='popup']//h4//a")

closeButton = driver.find_element_by_xpath('//*[@id="popup"]/footer/button')
closeButton.click()
sleep(2)

In [7]:
all_city_links_list = []

for eachCity in allCityLinks:
    all_city_links_list.append(eachCity.get_attribute('href'))


In [14]:
# all_city_links_list = ['http://www.schoolfinds.com/goa.html',
#  'http://www.schoolfinds.com/meghalaya.html',
#  'http://www.schoolfinds.com/nagaland.html',
#  'http://www.schoolfinds.com/lakshadweep.html',
#  'http://www.schoolfinds.com/pondicherry.html',
#  'http://www.schoolfinds.com/mizoram.html',
#  'http://www.schoolfinds.com/tripura.html',
#  'http://www.schoolfinds.com/andaman-nicobar.html']

In [17]:
all_city_links_list = ['http://www.schoolfinds.com/andaman-nicobar.html']

In [18]:
for eachCity in all_city_links_list:
    
    
    print("working for URL ", eachCity)
    driver.get(eachCity)
    sleep(5)
    
    noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
    print("Total no of schools are : ", noOfSchoolsTotal)
    
    noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
    print("possible number of ols estimated : ", noOfOlsEstimated)
    
    print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
    
    for i in range(noOfOlsEstimated):   # cover ols created
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        sleep(3)
        
        
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
        
        
        
    if (noOfSchoolsTotal < 10):
        print("no of schools are less than 10")
        driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
        
        noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
        print()
        print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
        for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
            try:
                schoolAboutDict = {}

                schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                schoolAboutDict['schoolName'] = schoolName

                schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                schoolAboutDict['schoolAddress'] = schoolAddressRefined

                schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                schoolAboutDict['schoolEmail'] = schoolEmail

                schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                schoolAboutDict['schoolContact'] = schoolContact

                schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                schoolAboutDict['schoolURL'] = schoolURL
                
                if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                    print("Already Present in the db")
                    continue

                collection_schoolsFindsPass.insert_one(schoolAboutDict)
                sleep(2)
                
                print("School Details : ", schoolAboutDict)
                print()
                
            except:
                
                print("Error for the head url city : ", eachCity)
                
                schoolDictFail = {}
                schoolDictFail['Error City URL'] = str(eachCity)
                
                if len(str(schoolName) > 0):
                    schoolDictFail['Error School Name'] = str(schoolName)
                
                collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
    else:
        
        if (noOfSchoolsTotal > noOfOlsEstimated*10):   # 196 > 190 
            print("WE NEED AN EXTRAA ITERATION!!!!  AS  :", noOfSchoolsTotal," > " ,noOfOlsEstimated*10)
        
            for i in range(1, noOfOlsEstimated + 2):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19,20

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        else:
            print("schools can be exact multiples of 10 ")
            
            for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        

working for URL  http://www.schoolfinds.com/andaman-nicobar.html
Total no of schools are :  68
possible number of ols estimated :  6
scrolling to page end!!! : for  6  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 68  >  60
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'GOVT SR SEC SCHOOL', 'schoolAddress': ': school line junglighat p.o pt blair south andaman a & n islands -744103', 'schoolEmail': 'Email: rnr@andaman.tn.nic.in', 'schoolContact': 'Contact Number: 03192 -32836', 'schoolURL': 'http://ww

School Inserted with success
School Details :  {'schoolName': 'GOVT. SECONDARY SCHOOL', 'schoolAddress': ': laxmipur nth andaman a & n islands -744202', 'schoolEmail': 'Email: rnr@andaman.tn.nic.in', 'schoolContact': 'Contact Number: 03192 -271390', 'schoolURL': 'http://www.schoolfinds.com/govt-secondary-school-16308.html', '_id': ObjectId('5b8a6d33edf2ee2c34fc8442')}

-------------------------STARTING FOR OL---------------------------- :  4


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'GOVT. SECONDARY SCHOOL', 'schoolAddress': ': keralapuram diglipur aerial bay po nth andaman a & n islands -744202', 'schoolEmail': 'Email: rnr@andaman.tn.nic.in', 'schoolContact': 'Contact Number: 03192 -73216', 'schoolURL': 'http://www.schoolfinds.com/govt-secondary-school-16307.html', '_id': ObjectId('5b8a6d38edf2ee2c34fc8443')}

School Inserted with success
School Details :  {'schoolName': 'GOVT. SECONDARY SCHOOL', 'schoolAddress': ': PASCHIM SAGAR


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'GOVT SECONDARY SCHOOL', 'schoolAddress': ': ram nagar post digilipur via nth andaman a & n islands -744202', 'schoolEmail': 'Email: rnr@andaman.tn.nic.in', 'schoolContact': 'Contact Number: 03192 -282618', 'schoolURL': 'http://www.schoolfinds.com/govt-secondary-school-16294.html', '_id': ObjectId('5b8a6d81edf2ee2c34fc8457')}

School Inserted with success
School Details :  {'schoolName': 'GOVT SECONDARY SCHOOL', 'schoolAddress': ': MILETILAK VIA WIMBERLYGUNJ SOUTH ANDAMAN A & N ISLANDS -744206', 'schoolEmail': 'Email: rnr@andaman.tn.nic.in', 'schoolContact': 'Contact Number: 03192 -281576', 'schoolURL': 'http://www.schoolfinds.com/govt-secondary-school-16295.html', '_id': ObjectId('5b8a6d85edf2ee2c34fc8458')}

School Inserted with success
School Details :  {'schoolName': 'GOVT SR SEC SCHOOL', 'schoolAddress': ': MAYABUNDER NORTH ANDAMAN A & N ISLANDS -744204', 'schoolEmail': 'Email: rnr@and